In [104]:
import pandas as pd

df = pd.read_csv("spotify-2023.csv", encoding='ISO-8859-1')

df.head(5)


,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"Latto, Jung Kook",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,Myke Towers,1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,Olivia Rodrigo,1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,Taylor Swift,1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,Bad Bunny,1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6


# Dealing with Null/Missing Values #

In [77]:
# Checking for null values in "in_shazam_charts" column
print(df['in_shazam_charts'].isnull().sum())

# drop shazam charts because of missing values / unnecessary for our analysis
df = df.drop(columns=['in_shazam_charts'], axis=1)

50


In [100]:
# find the missing values in the dataset in the key column
missing_values_sum = df['key'].isna().sum()
print(missing_values_sum)

# change NaN values to "Multiple Keys"
df['key'] = df['key'].fillna('Multiple Keys')

print(df['key'].value_counts(dropna=False))



0
C#               120
G                 96
Multiple Keys     95
G#                91
F                 89
B                 81
D                 81
A                 75
F#                73
E                 62
A#                57
D#                33
Name: key, dtype: int64


# Dealing With DataTypes #

In [73]:
# Check the datatypes and change if necessary
print(df.dtypes)

## Check for invalid Stream Values. 
for stream in df['streams']:
    try:
        int(stream)
    except ValueError:
        print(stream + ' is not an integer Value')

# delete line 576 row of the csv file because the streams value is a misinput, and without the actual streams count we can't impute
df = df[df['streams'] != 'BPM110KeyAModeMajorDanceability53Valence75Energy69Acousticness7Instrumentalness0Liveness17Speechiness3']
df['streams'] = df['streams'].apply(int)

track_name              object
artist(s)_name          object
artist_count             int64
released_year            int64
released_month           int64
released_day             int64
in_spotify_playlists     int64
in_spotify_charts        int64
streams                 object
in_apple_playlists       int64
in_apple_charts          int64
in_deezer_playlists     object
in_deezer_charts         int64
bpm                      int64
key                     object
mode                    object
danceability_%           int64
valence_%                int64
energy_%                 int64
acousticness_%           int64
instrumentalness_%       int64
liveness_%               int64
speechiness_%            int64
dtype: object
BPM110KeyAModeMajorDanceability53Valence75Energy69Acousticness7Instrumentalness0Liveness17Speechiness3 is not an integer Value


In [98]:
## Check for invalid Stream Values. 
invalid_streams = []
for stream in df['in_deezer_playlists']:
    try:
        int(stream)
    except ValueError:
        invalid_streams.append(stream)

# Check to see if there are any strings that we cannot convert to integers
print(len(invalid_streams))

# Print the first 5 elements of the array to see what the strings look like
print(invalid_streams[:5])

# Remove all commas in the strings (2,445 -> 2445) so we can cast to ints
df['in_deezer_playlists'].replace(',','', regex=True, inplace=True)

## Check for invalid Stream Values Again. 
invalid_streams = []
for stream in df['in_deezer_playlists']:
    try:
        int(stream)
    except ValueError:
        invalid_streams.append(stream)

# Check to see if there are any more invalid strings that we cannot convert to integers
print(len(invalid_streams))
df['in_deezer_playlists'] = df['in_deezer_playlists'].apply(int)



79
['2,445', '3,394', '3,421', '4,053', '1,056']
0


In [105]:
# Define a function that converts a string into a set of artists rather than array or single string
def get_artist_names(artist_string):
    if (type(artist_string) == str):
        return {artist_string}
    elif (type(artist_string) == list):
        return set(artist_string)

# Apply the function to the artist(s)_name column
df['artist(s)_name'] = df['artist(s)_name'].apply(get_artist_names)

# print head to inspect the data
df.head(5)

,track_name,artist(s)_name,artist_count,released_year,released_month,released_day,in_spotify_playlists,in_spotify_charts,streams,in_apple_playlists,...,bpm,key,mode,danceability_%,valence_%,energy_%,acousticness_%,instrumentalness_%,liveness_%,speechiness_%
0,Seven (feat. Latto) (Explicit Ver.),"{Latto, Jung Kook}",2,2023,7,14,553,147,141381703,43,...,125,B,Major,80,89,83,31,0,8,4
1,LALA,{Myke Towers},1,2023,3,23,1474,48,133716286,48,...,92,C#,Major,71,61,74,7,0,10,4
2,vampire,{Olivia Rodrigo},1,2023,6,30,1397,113,140003974,94,...,138,F,Major,51,32,53,17,0,31,6
3,Cruel Summer,{Taylor Swift},1,2019,8,23,7858,100,800840817,116,...,170,A,Major,55,58,72,11,0,11,15
4,WHERE SHE GOES,{Bad Bunny},1,2023,5,18,3133,50,303236322,84,...,144,A,Minor,65,23,80,14,63,11,6
